# Project \#3 - Face generation

### Deep Learning course -  A.Y. 2019-2020

Students:
- Simone Gayed Said
- Pierpasquale Colagrande


## Import of fundamental libraries
Herw we import fundamental libraries as TensorFlow, Numpy etc.

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)
"""
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)"""

Tensorflow version 2.2.0


"\ntry:\n  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection\n  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])\nexcept ValueError:\n  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')\n\ntf.config.experimental_connect_to_cluster(tpu)\ntf.tpu.experimental.initialize_tpu_system(tpu)\ntpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)"

In [0]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Flatten, Dense, Reshape, Concatenate, Lambda, Concatenate, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
import numpy as np
import matplotlib.pyplot as plt
import os


## Input pipeline

Our input data is stored on Google Cloud Storage. To more fully use the parallelism TPUs offer us, and to avoid bottlenecking on data transfer, we've stored our input data in TFRecord files, 2025 images per file.

Below, we make heavy use of `tf.data.experimental.AUTOTUNE` to optimize different parts of input loading.


In [0]:
BATCH_SIZE = 128
BUFFER_SIZE = 60000
ORIGINAL_IMAGE_SIZE = (218, 178, 3)
TARGET_IMAGE_SIZE = (64, 64, 3)
INTERMEDIATE_SIZE = 3072
ATTRIBUTES_SIZE = 40
LATENT_DIM = 64

In [0]:
AUTO = tf.data.experimental.AUTOTUNE

gcs_pattern = 'gs://celeba-test/tfrecord_*.tfrec'

filenames = tf.io.gfile.glob(gcs_pattern)

def parse_attribute_list(example):
  features = {
      "names": tf.io.FixedLenFeature([], tf.string),
  }

  example = tf.io.parse_single_example(example, features)
  attributes_names = example['names']
  return attributes_names

def get_names():
  record = tf.data.TFRecordDataset('gs://celeba-test/attribute_list.tfrec')
  attributes = record.map(parse_attribute_list)
  att_names = next(attributes.as_numpy_iterator()).decode("utf-8")
  att_names_list = [elem.strip()[1:-1] for elem in att_names.split(',')]
  return att_names_list

att_names_list = get_names()

feature_dict = {
      "image": tf.io.FixedLenFeature([], tf.string),
      "labels": tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
  }

def parse_tfrecord(example):
  features = feature_dict
  example = tf.io.parse_single_example(example, features)
  decoded = tf.image.decode_image(example['image'])  
  normalized = tf.cast(decoded, tf.float32) / 255.0 # convert each 0-255 value to floats in [0, 1] range
  image_tensor = tf.reshape(normalized, [ORIGINAL_IMAGE_SIZE[0], ORIGINAL_IMAGE_SIZE[1], ORIGINAL_IMAGE_SIZE[2]])
  image_tensor = tf.image.resize(image_tensor[45:173,25:153], (TARGET_IMAGE_SIZE[0], TARGET_IMAGE_SIZE[1])) # crop and reshape the image 
  labels = example['labels']
  return  {"encoder_input":image_tensor,"labels": tf.cast(labels,tf.float32)}


def load_dataset(filenames):
  # Read from TFRecords. For optimal performance, we interleave reads from multiple files.
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)


def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):

  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTO)

  return ds

image_ds = load_dataset(filenames)
training_dataset = prepare_for_training(image_ds)

Let's take a peek at the dataset we've created:

In [0]:
def show_batch(image_batch):
  plt.figure(figsize=(10,10))
  for n in range(25):
    ax = plt.subplot(5,5,n+1)
    plt.imshow(image_batch[n])
    plt.axis('off')

In [0]:
image_batch = next(iter(training_dataset))
show_batch(image_batch["encoder_input"].numpy())

## Network model
Here, we build the network model.

In [0]:
class Sampling(Layer):
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [0]:
"""LOSS_FACTOR = 10000
def r_loss(y_true, y_pred):
  return K.mean(K.square(y_true - y_pred), axis = [1,2,3])

def kl_loss(y_true, y_pred):
  kl_loss =  -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = 1)
  return kl_loss

def total_loss(y_true, y_pred):
  return LOSS_FACTOR*r_loss(y_true, y_pred) + kl_loss(y_true, y_pred)"""

In [0]:
def create_vae():
  # Define encoder model.
  original_inputs = Input(shape=TARGET_IMAGE_SIZE, name='encoder_input')
  labels = Input(shape = (ATTRIBUTES_SIZE,), name='labels')
  shape_before_flattening = K.int_shape(original_inputs)[1:] 
  x = Flatten()(original_inputs)
  xy = Concatenate()([x, labels])
  h = Dense(INTERMEDIATE_SIZE, activation='relu')(xy)
  z_mean = Dense(LATENT_DIM, name='z_mean')(h)
  z_log_var = Dense(LATENT_DIM, name='z_log_var')(h)
  z = Sampling()((z_mean, z_log_var))


  decoder_h = Dense(INTERMEDIATE_SIZE, activation='relu')
  decoder_out = Dense(np.prod(TARGET_IMAGE_SIZE), activation='sigmoid')
  # Define decoder model.
  zy = Concatenate()([z,labels])
  dec1 = decoder_h(zy)
  x_hat = decoder_out(dec1)
  x_hat =  Reshape(shape_before_flattening)(x_hat)

  #rec_loss = original_dim * metrics.binary_crossentropy(x, x_hat)
  #kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
  #vae_loss = K.mean(rec_loss + kl_loss)

  # Add KL divergence regularization loss.
  kl_loss = - 0.5 * tf.reduce_mean(
      z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)

  vae = Model(inputs=[original_inputs,labels], outputs=[x_hat], name = "vae")
  vae.add_loss(kl_loss)
  return vae

vae = create_vae()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer,loss=tf.keras.losses.MeanSquaredError())

vae.summary()

In [0]:
vae.fit(training_dataset, epochs=20, steps_per_epoch=202599//BATCH_SIZE, verbose=1)